# PHYS 481 Assignment 4: Regression and Machine Learning

On D2L, there are 2 datasets: daily sunspot number since 1818 (SN_d_tot_V2.0.csv) and geomagnetic indices (Kp and Ap) since 1932 (Kp_def.zip). The sunspot number data are from the World Data Center SILSO, Royal Observatory of Belgium, Brussels (https://www.sidc.be/SILSO/datafiles), while the geomagnetic data are from the GFZ German Research Centre for Geosciences, Potsdam, Germany (https://datapub.gfz-potsdam.de/download/10.5880.Kp.0001/). There is also a PDF description for the geomagnetic data (kp_index_data_description_20210311.pdf). Download the data from D2L to a local directory and unzip the zip file.

**Data References:**

Matzka, J., Bronkalla, O., Tornow, K., Elger, K., Stolle, C., 2021. "Geomagnetic Kp index". V. 1.0. GFZ Data Services. https://doi.org/10.5880/Kp.0001

SILSO World Data Center, 2024. "The International Sunspot Number", International Sunspot Number Monthly Bulletin and online catalogue, Royal Observatory of Belgium, avenue Circulaire 3, 1180 Brussels, Belgium, http://www.sidc.be/silso/

## Question 1: Look at the data before you use it

From the sunspot number CSV file, extract the date (first 3 columns) and the sunspot number (5th column). From the Kp_def files, extract the date (first 6 characters) and the daily Ap index (characters 56, 57 and 58, counting the first character as "1"). Trim the sunspot dataset to start at the same day as the Ap dataset (1932-01-01), check that the resulting datasets are the same length, and plot both datasets. You should see a clear 11-year period in the sunspot data and a less-clear 11-year period in Ap.

**HINT**: You may wish to store the date as a numpy datetime64 type. For example:

    date = np.datetime64(f"{year:04d}-{month:02d}-{day:02d}").

This will make it easier to display the data without worrying about things like leap years.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

def load_Ap_data(directory):
    """
    Load Ap index data from a specified directory.
    This function reads Ap index data files for the years 1932 to 2024 from the given directory.
    Each file is expected to be named in the format "Kp_def{year}.wdc". The function extracts
    the date and Ap index value from each file, skipping lines that start with a "#".
    Parameters:
        directory (str): The path to the directory containing the Ap index data files.
    Returns:
        dates (numpy.ndarray): An array of dates corresponding to the Ap index values.
        data (numpy.ndarray): An array of Ap index values.
    """
    dates = []
    data = []
    for year in range(1932, 2025):
        filename = f"Kp_def{year}.wdc"
        filepath = os.path.join(directory, filename)
        if os.path.exists(filepath):
            with open(filepath, 'r') as file:
                for line in file:
                    if line[0] == "#":
                        continue
                    date = line[:6]
                    date = date.replace(' ', '0')
                    date = np.datetime64(f"{year}-{date[2:4]}-{date[4:6]}")
                    dates.append(date)
                    value = line[55:58].strip()
                    data.append(np.float64(value))
    return np.squeeze(dates),np.squeeze(data)

def load_sunspot_data(directory):
    # Code and comments go here

def plot_q1(dates_sn, sn,dates_Ap, Ap):
    # Code and comments go here

# Load data

# Clip sunspot data time range to match Ap data

# Plot data


# Question 2: Linear regression

The sunspot number is a measure of the magnetic activity of the sun and the solar wind. The Ap index is a measure of electrical currents generated in Earth's upper atmosphere in response to the solar wind. (The same electrical currents are responsible for the aurora.)  The solar wind takes a few days to get from the sun to the Earth, so there's no reason to suspect the Ap index varies on a day-to-day basis in response to the sunspot number, but the plot from Q1 already suggests there's some correlation over a longer period. Let's try averaging over the 27-day solar rotation period and performing a linear regression.

1. Filter (smooth) the Ap and sunspot data with a 27-day boxcar average.
2. Plot a heatmap of the joint probability distribution ("2d histogram") of the smoothed sunspot number (x) and Ap index (y). Normalize each vertical column in the 2d histogram (i.e. each slice of roughly-constant sunspot number) to sum to one. Include the marginal distributions ("1d histograms"). There is code in the template to assist you.
3. Using a package of your choice (numpy, scipy, scikit-learn, etc.), fit a linear least-squares fit to Ap versus sunspot number. Calculate $R^2$ and the root mean squared error (RMSE). Add the least-squares line to the plot of the joint probability function, with a label giving the equation of the line of best fit, $R^2$ and the RMSE.


In [ ]:
def joint_pdf_plot(Ap,sn):
    """
    Plots the joint probability density function (PDF) of two variables, Ap and sn, along with their marginal distributions.
    Parameters:
        Ap (array-like): Array of values representing the Ap index.
        sn (array-like): Array of values representing the sunspot number.
    Returns:
        axs (numpy.ndarray): Array of matplotlib Axes objects.
    """
    # Setup axes
    fig, axs = plt.subplots(2, 2, figsize=(10,8), gridspec_kw={'height_ratios': [4,1], 'width_ratios': [1,4]})

    # 2D histogram of the joint probability of Ap and sn
    nbins=50
    h = np.histogram2d(Ap, sn, bins=nbins)
    h_normalized = h[0] / h[0].sum(axis=0, keepdims=True)
    axs[0,1].imshow(h_normalized, origin='lower', aspect='auto', extent=[h[2][0], h[2][-1],h[1][0], h[1][-1]],cmap='jet', norm=plt.Normalize(0, 0.2))
    plt.colorbar(axs[0,1].images[0], ax=axs[0,1], orientation='vertical', label='Normalized Occurrence Frequency')
    axs[0, 1].set_xlabel('Sunspot Number')
    axs[0, 1].set_ylabel('Ap index')

    # 1D histogram of sn
    axs[1, 1].hist(sn, bins=50, alpha=0.5, color='blue')
    axs[1, 1].set_ylabel('Frequency')
    axs[1, 1].set_xlim(axs[0, 1].get_xlim())
    axs[1, 1].set_position([axs[0, 1].get_position().x0, axs[1, 1].get_position().y0, axs[0, 1].get_position().width, axs[1, 1].get_position().height])

    # 1D histogram of Ap
    axs[0,0].hist(Ap, bins=50, alpha=0.5, color='blue', orientation='horizontal')
    axs[0,0].invert_xaxis()
    axs[0,0].set_xlabel('Frequency')
    axs[0,0].set_ylim(axs[0,1].get_ylim())

    # Remove the empty plot
    fig.delaxes(axs[1, 0])

    return axs

def joint_pdf_with_linear_fit(Ap,sn):
    """ 
    """
    # Plot the joint PDF
    axs=joint_pdf_plot(Ap,sn)

    # Perform linear regression using numpy

    # Plot the linear fit

    return axs


# Filter the data

# Plot joint PDF with a linear fit
joint_pdf_with_linear_fit(Ap_smoothed,sn_smoothed)


# Question 3: Polynomial Fits and Under- and Over-fitting

The linear fit in the previous question seems to miss a feature at low sunspot number, where the Ap data trends to zero. Let's see if we can pull that out with a nonlinear fit.

1. Split the data into a training set consisting of the first 70% of the data points and a validation set consisting of the remainder.
2. Fit the training set using polynomials from order one through order 20. (Use Polynomial.fit from numpy.polynomial.polynomial or a similar package.)
3. Plot the RMSE on the training set and the RMSE on the validation set as a function of polynomial order.
4. Choose the optimal order for the polynomial fit (i.e. the polynomial fit that best reduces the RMSE on the **validation** set)
5. Plot the joint probability distribution with:
    - the linear fit
    - the optimal polynomial fit 
    - the polynomial fit with order 20

In [ ]:
from numpy.polynomial.polynomial import Polynomial

def find_best_polynomial_order(Ap_train,Ap_test,sn_train,sn_test):
    """
    Plots the root mean squared error (RMSE) as a function of polynomial order for both the training and testing sets.
    The best polynomial order for fitting the Ap index data can be read off the plots.
    Parameters:
        Ap_train (array-like): Array of Ap index values for the training set.
        Ap_test (array-like): Array of Ap index values for the testing (validation) set.
        sn_train (array-like): Array of sunspot numbers for the training set.
        sn_test (array-like): Array of sunspot numbers for the testing set.
    """
    # Loop over polynomial orders to find the best fit

    # Plot RMSE as a function of polynomial order


def joint_pdf_with_polynomial_fit(Ap_smoothed,sn_smoothed,best_order):
    """
    Plots the joint probability density function (PDF) of two variables, Ap and sn, along with polynomial fits.
    """
    # Re-plot the joint PDF with a linear fit
    axs=joint_pdf_with_linear_fit(Ap_smoothed,sn_smoothed)

    # Plot 20th order polynomial fit

    # Plot best-order polynomial fit

    return axs


# Split data into training and testing sets

# First plot: RMSE vs polynomial order
find_best_polynomial_order(Ap_train,Ap_test,sn_train,sn_test)

# Best polynomial order will need to be determined from the plot
best_order = 19  # Change this to the best order found from the plot

# Second plot: Joint PDF with polynomial fits
joint_pdf_with_polynomial_fit(Ap_smoothed,sn_smoothed,best_order)


# Question 4: Neural Networks

As an alternative to the polynomial fitting from the previous question, use a neural network to fit the Ap index as a function of sunspot number. 
1. To reduce computation time and increase independence of the points, take every 27th point from the smoothed sunspot number and smoothed Ap to use as the x and y values for the regression.
2. Using the Multi-Layer Perceptron regressor (MLPRegressor) from scikit-learn, train a neural network to predict Ap as a function of sunspot number. The problem is relatively small and simple, so you don't need many layers (1 or 2 should suffice) or a large numbers of neurons (100 is plenty). The relu activation function should work well.
3. Calculate the RMSE for the fit.
4. Re-plot the joint probability distribution with the linear fit and the neural network fit.

[**NOTE**: To check for under- or over-fitting, we could split the data into training and validation sets like in the last question, and search for an optimal set of hyperparameters for the neural network (number of neurons, activation function, regularization parameter alpha, etc.). That's not required here.]

[**HINT**: This is very similar to the fit performed in the textbook: https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter25.03-Regression.html ]


In [ ]:
from sklearn.neural_network import MLPRegressor

# Choose evenly distributed points every 27 days


# Train neural network (multi-layer perceptron)

# Use neural network to predict Ap_smoothed

# Compute Mean Squared Error

# Re-plot the joint PDF with a linear fit
